In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import asyncpraw

In [4]:
reddit = await asyncpraw.Reddit(
    client_id=os.environ["REDDIT_CLIENT_ID"],
    client_secret=os.environ["REDDIT_CLIENT_SECRET"],
    # password=os.environ.get("REDDIT_PASSWORD"),
    user_agent=os.environ["USER_AGENT"]
    #, username=os.envirton.get("REDDIT_USERNAME")
).__aenter__()

# reddit.read_only = True|False (if using OAuth, default=False if OAuth is used)

### Ratelimits

In [5]:
print(reddit.auth.limits['used']) # if connected to OAuth

None


In [6]:
print(reddit.auth.limits['remaining']) # if connected to OAuth

None


In [7]:
print(reddit.auth.limits['reset_timestamp']) # if connected to OAuth

None


### Data retrieval

In [8]:
wsb = await reddit.subreddit("wallstreetbets")

In [16]:
submission_df = []

In [17]:
submissions = wsb.new(limit=2) # Must recreate for each async iteration
async for submission in submissions:
    await submission._fetch() # Necessary line
    submission_comments = []
    i = 0
    async for comment in submission.comments:
        if i != 0:
            submission_comments.append(comment.body)
        i += 1
    submission_df.append([submission.title, submission.url, submission.selftext, submission_comments])

In [18]:
pd.DataFrame(submission_df)

,0,1,2,3
0,Insider crypto hyper wallet trade,https://www.reddit.com/gallery/1ou26qf,🚨 Hyperliquid wallet 0xb317d2bc2d3d2df5fa441b5...,[]
1,5600% Gains with Options,https://www.reddit.com/r/wallstreetbets/commen...,Been investing with bi-weekly every time i rec...,"[Good job, now buy SPY with 80% and do it agai..."


In [89]:
await reddit.close()

In [ ]:
subreddit = await reddit.subreddit("wallstreetbets+finance+investing")
async for submission in subreddit.sticky():
    print(submission)

In [81]:
subreddit = await reddit.subreddit("wallstreetbets")
thread = await subreddit.sticky(2) # What are your Moves tomorrow

In [ ]:
await thread.comments.replace_more(limit=None)

In [ ]:
comments = thread.comments.list()

In [ ]:
sticky_df = []
subreddit = await reddit.subreddit("wallstreetbets")
thread = await subreddit.sticky(2)
thread_comments = []
i = 0
async for comment in thread.comments:
    
    if i != 0:
        thread_comments.append(comment.children)
    i += 1
submission_df.append([thread.title, thread.url, thread.selftext, thread_comments])

AttributeError: 'Comment' object has no attribute 'children'